In [1]:
import pandas as pd
import sys
from data_reader import DataReader, rename_columns
from tqdm import tqdm
import bisect
import json
import datetime

import json
import logging
import os
import sys
from datetime import date

from finder import find_layers
from actual_perf import get_actual_perf
from writexl import write_layers, write_act_perf
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                 'cygwin': '\\',
                 'darwin': '/',
                 'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])


# очистка папки output_folder
def clear_out_folder(output_folder):
    files = os.listdir(output_folder)
    for f in files:
        path_dir = replace_slash(output_folder + "\\" + f)
        os.remove(path_dir)


def get_year(conf_perf_year):
    if conf_perf_year == '':
        return None
    try:
        return date(year=int(conf_perf_year), month=1, day=1)
    except:
        return None

def rename_columns(df):
    col_names = {'well': '', 'top': '', 'bot': '',
                 'soil': '', 'date': '', 'type': ''}
    for column in df.columns.values:
        if ('скв' in column) or ('skw_nam' in column) or (column == 'skw'):
            col_names['well'] = column
        elif ('verh' in column) or ('krow' == column) or ('верх' in column) or ('perf1_t' in column):
            col_names['top'] = column
        elif ('niz' in column) or ('podosh' in column) or ('низ' in column) or ('perf2_t' in column):
            col_names['bot'] = column
        elif ('nnas' in column) or ('н_нас' in column):
            col_names['soil'] = column
        elif ('дата_перф' in column) or (column == 'dat'):
            col_names['date'] = column
        elif ('цель' in column) or ('_cel' in column):
            col_names['type'] = column
    df.rename(columns={col_names['bot']: 'bot', col_names['top']: 'top',
                       col_names['well']: 'well', col_names['soil']: 'soil',
                       col_names['date']: 'date', col_names['type']: 'type'},
              inplace=True)
    col_names_set = set(df.columns)
    df.drop(columns=list(col_names_set.difference(col_names.keys())),
            inplace=True)

In [3]:
input_folder = "input_data"
out_folder = "output_data"
output_path_l = replace_slash(out_folder + "\\" + "non_perf_layers.xlsx")
output_path_a = replace_slash(out_folder + "\\" + "act_perf.xlsx")

if not os.path.exists(out_folder):
    os.makedirs(out_folder)
else:
    clear_out_folder(out_folder)

out_path_log = replace_slash(out_folder + "\\" + "Report.txt")
logging.basicConfig(format=u'%(levelname)-8s : %(message)s', filename=out_path_log, filemode='w')

with open("config.json", 'r') as f:
    try:
        conf = json.load(f)
        SOIL_CUT = float(conf["SOIL_CUT"])
        perf_path = conf["perf_path"]
        fes_path = conf["fes_path"]
        act_perf_year = get_year(conf["act_perf_year"])
    except BaseException as e:
        logging.error("Error loading config file. " + str(e))
        sys.exit()
perf_path = replace_slash(input_folder + '\\' + perf_path)
fes_path = replace_slash(input_folder + '\\' + fes_path)

dr = DataReader()
try:
    perf_ints = dr.perf_reader(perf_path)
except BaseException as e:
    logging.error("Error loading perf file. " + str(e))
    sys.exit()
try:
    fes_dict = dr.fes_reader(fes_path)
except BaseException as e:
    logging.error("Error loading fes file. " + str(e))
    sys.exit()

perf_rig_diff, rig_perf_diff = dr.well_diff()
if len(perf_rig_diff) > 0:
    logging.warning("These wells in perf file are absent in rigis "
                    + str(perf_rig_diff))
if len(rig_perf_diff) > 0:
    logging.warning("These wells in rigis file are absent in perf "
                    + str(rig_perf_diff))

try:
    lost_layers = find_layers(perf_ints, fes_dict, SOIL_CUT)
except BaseException as e:
    logging.error("Error while finding layers " + str(e))
    sys.exit()

try:
    act_perf = get_actual_perf(perf_ints, act_perf_year)
except BaseException as e:
    logging.error("Error while getting the actual perforation " + str(e))
    sys.exit()

started reading perf xl
done reading perf xl and started processing perf data
started reading fes xl


100%|██████████| 1579/1579 [00:00<00:00, 41957.12it/s]

done reading fes xl
done processing data


In [4]:
wells1 = list(dr.sl_wells)

In [5]:
dr.sl_wells = set()

In [6]:
wells2 = dr.sl_wells

In [8]:
act_df = pd.read_json(json.dumps(act_perf))

act_df2 =  pd.read_json('perf2.json')

act_df2.rename(columns=lambda x: x.lower().strip(), inplace=True)

rename_columns(act_df2)

act_df2['well'] = act_df2['well'].apply(dr.well_renaming)
act_df = act_df[act_df['perf_type'] == 1]
act_df2.sort_values(by=['well', 'top', 'bot'], ascending=False, inplace=True)
act_df.sort_values(by=['well', 'top', 'bot'], ascending=False, inplace=True)
act_df.drop(act_df[act_df['top'] == act_df['bot']].index, inplace=True)
act_df2.drop(act_df2[act_df2['top'] == act_df2['bot']].index, inplace=True)
act_df.reset_index(drop=True, inplace=True)
act_df2.reset_index(drop=True, inplace=True)
del_idxs = []
for i in tqdm(range(len(act_df2) - 1)):
    if ((act_df2.loc[i, 'top'] == act_df2.loc[i + 1, 'bot']) or (act_df2.loc[i, 'top'] == act_df2.loc[i + 1, 'top'])) and (act_df2.loc[i, 'well'] == act_df2.loc[i + 1, 'well']):
        act_df2.loc[i + 1, 'bot'] = act_df2.loc[i, 'bot']
        del_idxs.append(i)
act_df2.drop(del_idxs, inplace=True)
act_df2.reset_index(drop=True, inplace=True)

del_idxs = []
for i in tqdm(range(len(act_df2) - 1)):
    if (act_df2.loc[i, 'bot'] <= act_df2.loc[i + 1, 'bot']) and (act_df2.loc[i, 'well'] == act_df2.loc[i + 1, 'well']):
        del_idxs.append(i)
act_df2.drop(del_idxs, inplace=True)
act_df2.reset_index(drop=True, inplace=True)

del_idxs = []
for i in tqdm(range(len(act_df) - 1)):
    if (act_df.loc[i, 'bot'] <= act_df.loc[i + 1, 'bot']) and (act_df.loc[i, 'well'] == act_df.loc[i + 1, 'well']):
        del_idxs.append(i)
act_df.drop(del_idxs, inplace=True)
act_df.reset_index(drop=True, inplace=True)

del_idxs = []
for i in tqdm(range(len(act_df) - 1)):
    if ((act_df.loc[i, 'top'] == act_df.loc[i + 1, 'bot']) or (act_df.loc[i, 'top'] == act_df.loc[i + 1, 'top'])) and (act_df.loc[i, 'well'] == act_df.loc[i + 1, 'well']):
        act_df.loc[i + 1, 'bot'] = act_df.loc[i, 'bot']
        del_idxs.append(i)
act_df.drop(del_idxs, inplace=True)
act_df.reset_index(drop=True, inplace=True)

                    
len(list(set(act_df['well'].unique()).difference(act_df2['well'].unique())))

len(list(set(act_df2['well'].unique()).difference(act_df['well'].unique())))

act_df_intersec = act_df[act_df['well'].isin(act_df2['well'].unique())]

act_df_diff = act_df_intersec[act_df_intersec['perf_type'] == 1][['well', 'top', 'bot']]

diff = act_df_diff[~act_df_diff.apply(tuple,1).isin(act_df2.apply(tuple,1))]
diff2 = act_df2[~act_df2.apply(tuple,1).isin(act_df_diff.apply(tuple,1))]

100%|██████████| 4193/4193 [00:00<00:00, 27079.82it/s]


In [9]:
print(len(diff))
print(len(diff2))

74
12


In [10]:
diff3 = pd.concat([act_df2, act_df_diff]).drop_duplicates(keep=False)

In [11]:
print(len(diff))
print(len(diff2))
print(len(diff3))

74
12
86


In [ ]:
diff.head(20)

In [12]:
act_df_diff[act_df_diff['well']=='32787']

,well,top,bot
739,32787,1636.0,1639.0
740,32787,1630.3,1634.0
741,32787,1624.0,1628.0


In [13]:
act_df2[act_df2['well']=='32787']

,well,top,bot
721,32787,1636.0,1639.0
722,32787,1630.3,1634.0
723,32787,1624.0,1628.0


In [14]:
print(sorted(wells1))

['10829/1', '10829/2', '10877А/1', '10877А/2', '10897/1', '10897/2', '14933/1', '14933/2', '15286/2', '15632/1', '15632/2', '15693/1', '15693/2', '173/1', '173/2', '17761/1', '17761/2', '17765/1', '17765/2', '20176/1', '20176/2', '20201/1', '20201/2', '20224/1', '20224/2', '20306/1', '20306/2', '20334/1', '20334/2', '20336/2', '20337Д/1', '20337Д/2', '20341/1', '20341/2', '20451/1', '20451/2', '20613/1', '20613/2', '29589/1', '29589/2', '3187/1', '3187/2', '32642/2', '32665/1', '32665/2', '3451/1', '3451/2', '9517/1', '9517/2', '9526/1', '9526/2', '9555/1', '9616/1', '9616/2', '9616/3', '9618/1', '9618/2', '9636/1', '9636/2']


In [15]:
diff_wells = diff3['well'].unique()
print(len(diff_wells))
print(sorted(diff_wells))

60
['10756', '10792', '10800', '10801', '10839', '10840', '10843', '1205', '1214', '145', '14936', '14937', '14942', '14963', '152', '153', '154', '16555', '179', '199', '20113', '20147', '20174', '20186', '20211а', '20323', '20359', '20408', '20414', '20415', '20454', '20508', '20531', '20635', '20645', '20648', '206д', '216д', '231', '242', '262', '270', '294', '3056', '3100', '3123', '3128', '3133', '3179', '3184', '3192', '32693', '3446', '3469', '3488д', '3682д', '440д', '9517', '9525', '9624']


In [16]:
print(sorted(list(wells2)))

['10751Д/1', '10751Д/2', '10788/1', '10788/2', '10793/1', '10793/2', '10816А/1', '10816А/2', '10817/1', '10817/2', '10829/2', '10834А/1', '10834А/2', '10834Д/1', '10834Д/2', '10837/1', '10837/2', '10840/2', '10857/1', '10857/2', '10870/1', '10870/2', '10877А/2', '10892/1', '10892/2', '10897/2', '1205А/1', '1205А/2', '1207А/1', '1207А/2', '139Д/1', '139Д/2', '14898/1', '14898/2', '14900/1', '14900/2', '14904/1', '14904/2', '14906/1', '14906/2', '14916/1', '14916/2', '14922/1', '14922/2', '14924/1', '14924/2', '14933/2', '14936/2', '14940/1', '14940/2', '14942/2', '14953/1', '14953/2', '14957/1', '14957/2', '14992/1', '14992/2', '15286/2', '15632/2', '15693/2', '15756Д/1', '15756Д/2', '158/1', '158/2', '169Д/1', '169Д/2', '173/2', '176/1', '176/2', '176Д/1', '176Д/2', '17761/2', '17765/2', '20118/1', '20118/2', '20129/1', '20129/2', '20136/1', '20136/2', '20157/1', '20157/2', '20163/1', '20163/2', '20176/2', '20188/1', '20188/2', '20194/1', '20194/2', '20201/2', '20202/1', '20202/2', '20

In [17]:
wells2

{'10751Д/1',
 '10751Д/2',
 '10788/1',
 '10788/2',
 '10793/1',
 '10793/2',
 '10816А/1',
 '10816А/2',
 '10817/1',
 '10817/2',
 '10829/2',
 '10834А/1',
 '10834А/2',
 '10834Д/1',
 '10834Д/2',
 '10837/1',
 '10837/2',
 '10840/2',
 '10857/1',
 '10857/2',
 '10870/1',
 '10870/2',
 '10877А/2',
 '10892/1',
 '10892/2',
 '10897/2',
 '1205А/1',
 '1205А/2',
 '1207А/1',
 '1207А/2',
 '139Д/1',
 '139Д/2',
 '14898/1',
 '14898/2',
 '14900/1',
 '14900/2',
 '14904/1',
 '14904/2',
 '14906/1',
 '14906/2',
 '14916/1',
 '14916/2',
 '14922/1',
 '14922/2',
 '14924/1',
 '14924/2',
 '14933/2',
 '14936/2',
 '14940/1',
 '14940/2',
 '14942/2',
 '14953/1',
 '14953/2',
 '14957/1',
 '14957/2',
 '14992/1',
 '14992/2',
 '15286/2',
 '15632/2',
 '15693/2',
 '15756Д/1',
 '15756Д/2',
 '158/1',
 '158/2',
 '169Д/1',
 '169Д/2',
 '173/2',
 '176/1',
 '176/2',
 '176Д/1',
 '176Д/2',
 '17761/2',
 '17765/2',
 '20118/1',
 '20118/2',
 '20129/1',
 '20129/2',
 '20136/1',
 '20136/2',
 '20157/1',
 '20157/2',
 '20163/1',
 '20163/2',
 '20176/2

In [18]:
pd.DataFrame(data={'скважины': diff_wells}).to_excel('diff_wells.xlsx', index=False)

In [19]:
diff_df = pd.DataFrame(columns=['source','well', 'top', 'bot'])

In [20]:
act_df2['source'] = 'армитс'
act_df_diff['source'] = 'local'

In [21]:
for well in diff_wells:
    diff_df = diff_df.append(act_df_diff[act_df_diff['well']== well])
    diff_df = diff_df.append(act_df2[act_df2['well']== well])    

In [22]:
diff_df.to_excel('diff_wells_info.xlsx', index=False)